In [1]:
import keras_tuner as kt
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype(np.float32)
x_train /= 255.0
x_test = x_test.astype(np.float32)
x_test /= 255.0

In [3]:
def model_builder(hp: kt.engine.hyperparameters.HyperParameters):
    model = tf.keras.Sequential()
    
    model.add(layers.Flatten(input_shape=(28, 28)))
    
    hp_units = hp.Int('units', min_value=16, max_value=512, step=16)
    model.add(layers.Dense(units=hp_units, activation='relu'))
    
    model.add(layers.Dropout(0.2))
    
    model.add(layers.Dense(10, activation='softmax'))
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [4]:
tuner = kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='tuner_dir',
    project_name='demo'
)

In [5]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [6]:
tuner.search(
    x_train,
    y_train,
    epochs=50,
    validation_split=0.2,
    callbacks=[early_stop]
)

Trial 30 Complete [00h 00m 23s]
val_accuracy: 0.9798333048820496

Best val_accuracy So Far: 0.9798333048820496
Total elapsed time: 00h 05m 11s
INFO:tensorflow:Oracle triggered exit


In [7]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [8]:
best_hps.get('units')

288